In [ ]:
# !pip install aiohttp

In [1]:
import pandas as pd
import requests
from dotenv import load_dotenv
import os
from yfpy.query import YahooFantasySportsQuery
from yfpy.models import Scoreboard, Matchup, YahooFantasyObject
from yahoo_oauth import OAuth2
import xml.etree.ElementTree as ET
from collections import defaultdict
from pprint import pprint

import sys
import os
from datetime import datetime, timedelta, timezone
import requests
from zoneinfo import ZoneInfo
from collections import defaultdict
import math
from pprint import pprint

# Add the project root directory to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), "../app"))
if project_root not in sys.path:
    sys.path.append(project_root)

from query import Query
from metrics import Metrics

In [2]:
# Use last season stats so there aren't any spoilers
current = False
league_id = "67269" if current else "97108"
game_id = 453 if current else 427
league_key = f"{game_id}.l.{league_id}"
query = await Query.create(league_key)

In [7]:
BASE_URL = "https://fantasysports.yahooapis.com/fantasy/v2"

In [11]:
def get_team_name(team_key):
    team_name = next((team["name"] for team in teams if team["team_key"] == team_key), 'Unknown Team')
    return team_name 

In [10]:
def get_players(player_keys):
    return [
        player 
        for i in range(int(len(player_keys) / 25) + 1)
        for player in query.query(
            f"https://fantasysports.yahooapis.com/fantasy/v2/league/{league_info.league_key}/players;player_keys={','.join(player_keys[i*25:min((i+1)*25, len(player_keys))])};start={i*25}/stats",
            ["league", "players"]
        )
    ]

In [ ]:
'''USES YFPY'''

def get_full_scoring_df():
    league_settings = query.get_league_settings()
    scoring_categories = league_settings.stat_categories.stats
    for i in range(len(scoring_categories)):
        scoring_categories[i] = scoring_categories[i].clean_data_dict()
    scoring_categories = pd.DataFrame(data=scoring_categories)

    scoring_modifiers = league_settings.stat_modifiers.stats
    scoring_modifiers = [scoring_modifiers[i].clean_data_dict() for i in range(len(scoring_modifiers))]

    scoring_settings = pd.merge(left=scoring_categories, right=pd.DataFrame(data=scoring_modifiers), on='stat_id', how='inner')
    return scoring_settings


def get_scoring_dict_by_stat_id(scoring_settings_df):
    return {str(scoring_settings_df['stat_id'][i]) : float(scoring_settings_df['value'][i]) for i in range(len(scoring_settings_df))}

SCORING_SETTINGS = get_full_scoring_df()
SCORING_MODIFIERS_BY_STAT_ID = get_scoring_dict_by_stat_id(SCORING_SETTINGS)

Draft Guru - Best/Worst Drafts

In [ ]:
'''
Draft Guru - Best/Worst Draft
Evaulated by summing points scored by all players drafted for each team
Returns list of total drafted points for each team sorted highest to lowest
'''
async def get_best_worst_drafts():
    url = f"/league/{query.league_key}/draftresults"
    response = await query.get_response(url)
    full_draft = response['league']['draft_results']
    full_draft = pd.DataFrame(data=full_draft, columns=['pick', 'round', 'team_key', 'player_key'])
    teams = query.get_teams()
    team_keys = tuple(teams.keys())
    team_drafts = {query.get_team_name_from_key(team_key) : full_draft[full_draft["team_key"] == team_key] for team_key in team_keys}
    ranked_drafts_list = []
    i=0
    for team, draft in team_drafts.items():
        stats = await query.get_players(player_keys=draft["player_key"])
        ranked_drafts_list.append(
            {
                "rank": 0,
                "image_url": query.teams[next((i for i, t in enumerate(query.teams) if t["name"] == team), None)]["team_logos"]["team_logo"]["url"],
                "main_text": team,
                "sub_text": "",
                "stat": 0
            }
        )
        i+=1
        for j in range(len(draft)):
            ranked_drafts_list[len(ranked_drafts_list)-1]["stat"] += round(float(stats[j]["player_points"][0]["total"]), 1)
    ranked_drafts_list = sorted(ranked_drafts_list, key=lambda item: list(item.items())[4][1], reverse=True)
    rank=1
    for draft in ranked_drafts_list:
        draft["rank"] = rank
        draft["stat"] = round(draft["stat"], 1)
        rank+=1
    
    return {"id": "best_worst_drafts",
            "data": ranked_drafts_list}

In [4]:
metrics = Metrics(query)
await metrics.get_best_worst_drafts()

{'id': 'best_worst_drafts',
 'data': [{'rank': 1,
   'image_url': 'https://s.yimg.com/cv/apiv2/default/nhl/nhl_1_t.png',
   'main_text': "Tim's Terrific Team",
   'sub_text': '',
   'stat': 5554.2},
  {'rank': 2,
   'image_url': 'https://s.yimg.com/cv/apiv2/default/nhl/nhl_5_t.png',
   'main_text': "Theodore's Poo Poo Hospital",
   'sub_text': '',
   'stat': 5315.4},
  {'rank': 3,
   'image_url': 'https://s.yimg.com/cv/apiv2/default/nhl/nhl_12_k.png',
   'main_text': 'Eric’s a Bitch Team',
   'sub_text': '',
   'stat': 5312.0},
  {'rank': 4,
   'image_url': 'https://s.yimg.com/cv/apiv2/default/nhl/nhl_6_e.png',
   'main_text': "eric's Awe-Inspiring Team",
   'sub_text': '',
   'stat': 5007.2},
  {'rank': 5,
   'image_url': 'https://s.yimg.com/cv/apiv2/default/nhl/nhl_8_a.png',
   'main_text': 'adam2',
   'sub_text': '',
   'stat': 5002.5},
  {'rank': 6,
   'image_url': 'https://s.yimg.com/cv/apiv2/default/nhl/nhl_9_c.png',
   'main_text': "Cyrus's Cursed Team",
   'sub_text': '',
   's

Mike Sillinger Award - Most Picked up/Dropped Player

In [ ]:
'''
Mike Sillinger Award
Evaluated by who was dropped the most
Returns Top 10
'''
async def get_most_dropped_players(self):
    url = f"/league/{self.query.league_key}/transactions"
    response = await self.query.get_response(url)
    transactions = response['league']['transactions']
    drops = {}
    missed = 0
    for transaction in transactions:
        if (transaction['type'] == 'drop') and transaction['status'] == 'successful':
            try:
                drops[transaction['players'][0]['player_key']][1] += 1
            except:
                drops[transaction['players'][0]['player_key']] = [transaction['players'][0]['name']['full'], 1]
        elif transaction['type'] == 'add/drop' and transaction['status'] == 'successful':
            try:
                drops[transaction['players'][1]['player_key']][1] += 1
            except:
                drops[transaction['players'][1]['player_key']] = [transaction['players'][1]['name']['full'], 1]

    drops = dict(sorted(drops.items(), key=lambda item: item[1][1], reverse=True))
    top_x = 10
    players = await query.get_players(list(drops.keys())[:top_x])   
    top_drops_list = [
        {
            "rank": i+1,
            "image_url": players[i]['image_url'],
            "main_text": list(drops.values())[i][0],
            "sub_text": "",
            "stat": list(drops.values())[i][1]
         } for i in range(top_x)
    ]
    
    return top_drops_list

In [137]:
metrics = Metrics(query)
await metrics.get_most_dropped_players()

[{'id': 'most_dropped',
  'data': [{'rank': 1,
    'image_url': 'https://s.yimg.com/iu/api/res/1.2/jKwQ9jvG0kHwy.T4LpPgeg--~C/YXBwaWQ9eXNwb3J0cztjaD0yMzM2O2NyPTE7Y3c9MTc5MDtkeD04NTc7ZHk9MDtmaT11bGNyb3A7aD02MDtxPTEwMDt3PTQ2/https://s.yimg.com/xe/i/us/sp/v/nhl_cutout/players_l/10162024/8290.png',
    'main_text': 'Matt Boldy',
    'sub_test': '',
    'stat': '7 add/drops'},
   {'rank': 2,
    'image_url': 'https://s.yimg.com/iu/api/res/1.2/Lwy0Z2iRU0WNAGYUUE3AUw--~C/YXBwaWQ9eXNwb3J0cztjaD0yMzM2O2NyPTE7Y3c9MTc5MDtkeD04NTc7ZHk9MDtmaT11bGNyb3A7aD02MDtxPTEwMDt3PTQ2/https://s.yimg.com/xe/i/us/sp/v/nhl_cutout/players_l/10022024/7569.png',
    'main_text': 'Ukko-Pekka Luukkonen',
    'sub_test': '',
    'stat': '6 add/drops'},
   {'rank': 3,
    'image_url': 'https://s.yimg.com/iu/api/res/1.2/p4WEbLzDBns54AxK0Z1nwA--~C/YXBwaWQ9eXNwb3J0cztjaD0yMzM2O2NyPTE7Y3c9MTc5MDtkeD04NTc7ZHk9MDtmaT11bGNyb3A7aD02MDtxPTEwMDt3PTQ2/https://s.yimg.com/xe/i/us/sp/v/nhl_cutout/players_l/10172024/5986.png',
    'mai

Closest Weekly Matchups, Biggest Blowouts, and Lopsided Rivalry

In [4]:
await query.get_matchups()

[{'week': '1',
  'week_start': '2023-10-10',
  'week_end': '2023-10-15',
  'status': 'postevent',
  'is_playoffs': '0',
  'is_consolation': '0',
  'is_tied': '0',
  'winner_team_key': '427.l.97108.t.1',
  'stat_winners': [{'stat_id': '1', 'winner_team_key': '427.l.97108.t.1'},
   {'stat_id': '2', 'winner_team_key': '427.l.97108.t.1'},
   {'stat_id': '4', 'winner_team_key': '427.l.97108.t.8'},
   {'stat_id': '8', 'winner_team_key': '427.l.97108.t.1'},
   {'stat_id': '11', 'winner_team_key': '427.l.97108.t.1'},
   {'stat_id': '14', 'winner_team_key': '427.l.97108.t.8'},
   {'stat_id': '31', 'winner_team_key': '427.l.97108.t.1'},
   {'stat_id': '32', 'winner_team_key': '427.l.97108.t.1'},
   {'stat_id': '19', 'is_tied': '1'},
   {'stat_id': '22', 'winner_team_key': '427.l.97108.t.1'},
   {'stat_id': '25', 'winner_team_key': '427.l.97108.t.8'},
   {'stat_id': '27', 'winner_team_key': '427.l.97108.t.8'}],
  'teams': [{'team_key': '427.l.97108.t.1',
    'team_id': '1',
    'name': 'adam2',
 

In [15]:
async def get_league_matchup_results_by_week(weeks: list[int]):
    weeks = ','.join(str(week) for week in weeks)
    url = f"/league/{league_key}/scoreboard;week={weeks}"
    response = await query.get_response(url)
    return response['league']['scoreboard']['matchups']

In [54]:
await get_league_matchup_results_by_week([1,2,3])

[{'week': '1',
  'week_start': '2023-10-10',
  'week_end': '2023-10-15',
  'status': 'postevent',
  'is_playoffs': '0',
  'is_consolation': '0',
  'is_tied': '0',
  'winner_team_key': '427.l.97108.t.1',
  'stat_winners': [{'stat_id': '1', 'winner_team_key': '427.l.97108.t.1'},
   {'stat_id': '2', 'winner_team_key': '427.l.97108.t.1'},
   {'stat_id': '4', 'winner_team_key': '427.l.97108.t.8'},
   {'stat_id': '8', 'winner_team_key': '427.l.97108.t.1'},
   {'stat_id': '11', 'winner_team_key': '427.l.97108.t.1'},
   {'stat_id': '14', 'winner_team_key': '427.l.97108.t.8'},
   {'stat_id': '31', 'winner_team_key': '427.l.97108.t.1'},
   {'stat_id': '32', 'winner_team_key': '427.l.97108.t.1'},
   {'stat_id': '19', 'is_tied': '1'},
   {'stat_id': '22', 'winner_team_key': '427.l.97108.t.1'},
   {'stat_id': '25', 'winner_team_key': '427.l.97108.t.8'},
   {'stat_id': '27', 'winner_team_key': '427.l.97108.t.8'}],
  'teams': [{'team_key': '427.l.97108.t.1',
    'team_id': '1',
    'name': 'adam2',
 

In [5]:
'''
Gets a dictionary with completed weekly matchup information
Used in Closed Matchups, Biggest Blowouts, and Lopsided Rivalry Awards
'''
async def get_matchup_data():
    url = f"/league/{league_key}"
    response = await query.get_response(url)
    league = response['league']
    start_week = int(league['start_week'])
    end_week = int(league['end_week'])
    weeks = list(range(start_week, end_week+1))
    weekly_matchup_data = await get_league_matchup_results_by_week(weeks)
    completed_matchups_data = []
    winning_team = 0
    losing_team = 1
    for matchup_num in range(len(weekly_matchup_data)):
        point_diff = round(float(weekly_matchup_data[matchup_num]['teams'][0]['team_points']['total']) - float(weekly_matchup_data[matchup_num]['teams'][1]['team_points']['total']), 2)
        if point_diff < 0:
            winning_team = 1
            losing_team = 0
        else:
            winning_team = 0
            losing_team = 1
        completed_matchups_data.append(
            {
                'team1_key' : weekly_matchup_data[matchup_num]['teams'][winning_team]['team_key'],
                'team1_name' : weekly_matchup_data[matchup_num]['teams'][winning_team]['name'],
                'team1_points' : float(weekly_matchup_data[matchup_num]['teams'][winning_team]['team_points']['total']),
                'team1_url' : weekly_matchup_data[matchup_num]['teams'][winning_team]['team_logos']['team_logo']['url'],
                'team2_key' : weekly_matchup_data[matchup_num]['teams'][losing_team]['team_key'],
                'team2_name' : weekly_matchup_data[matchup_num]['teams'][losing_team]['name'],
                'team2_points' : float(weekly_matchup_data[matchup_num]['teams'][losing_team]['team_points']['total']),
                'team2_url' : weekly_matchup_data[matchup_num]['teams'][losing_team]['team_logos']['team_logo']['url'],
                'point_diff' : abs(point_diff),
                'is_tied' : int(weekly_matchup_data[matchup_num]['is_tied']),
                'week' : int(weekly_matchup_data[matchup_num]['week']),
                'is_playoffs' : int(weekly_matchup_data[matchup_num]['is_playoffs']),
                'is_consolation' : int(weekly_matchup_data[matchup_num]['is_consolation'])
            }
        )
    return completed_matchups_data

In [8]:
'''
Closest Matchups
'''
async def get_closest_matchups():
    completed_matchups_data = await query.get_matchup_data()
    top_x = 10
    matchups_data_by_point_diff_ascen = sorted(completed_matchups_data, key=lambda x: abs(x['point_diff']))
    closest_matchups_list = [
            {
            "rank": i+1,
            "image_url": matchups_data_by_point_diff_ascen[i]['team1_url'],
            "main_text": f"{query.get_team_name_from_key(matchups_data_by_point_diff_ascen[i]['team1_key'])} ({matchups_data_by_point_diff_ascen[i]['team1_points']}) def. {query.get_team_name_from_key(matchups_data_by_point_diff_ascen[i]['team2_key'])} ({matchups_data_by_point_diff_ascen[i]['team2_points']})"
                if not matchups_data_by_point_diff_ascen[i]['is_tied'] else 
                f"{query.get_team_name_from_key(matchups_data_by_point_diff_ascen[i]['team1_key'])} and {query.get_team_name_from_key(matchups_data_by_point_diff_ascen[i]['team2_key'])} tied",
            "sub_text": "playoffs" if matchups_data_by_point_diff_ascen[i]['is_playoffs'] else "",
            "stat": matchups_data_by_point_diff_ascen[i]['point_diff']
            } for i in range(top_x)
        ]
    return {"id": "closest_matchups", "data": closest_matchups_list}


    # narrow_wins = {team_key : 0 for team_key in team_keys}
    # narrow_losses = {team_key : 0 for team_key in team_keys}
    # narrow_matchups = {team_key : 0 for team_key in team_keys}
#     if matchups_data_by_point_diff_ascen[i]['is_tied']:
#         print(f"{i+1}. {matchups_data_by_point_diff_ascen[i]['team1_name']} and {matchups_data_by_point_diff_ascen[i]['team2_name']} tied at {matchups_data_by_point_diff_ascen[i]['team1_points']}")
#     else:
#         print(f"{i+1}. {matchups_data_by_point_diff_ascen[i]['team1_name']} def. {matchups_data_by_point_diff_ascen[i]['team2_name']}")
#         print(f"Score: {matchups_data_by_point_diff_ascen[i]['team1_points']}-{matchups_data_by_point_diff_ascen[i]['team2_points']}") 
#         print(f"Point Differential: {abs(matchups_data_by_point_diff_ascen[i]['point_diff'])}")
#         print(f"Week: {matchups_data_by_point_diff_ascen[i]['week']}")
#         narrow_wins[matchups_data_by_point_diff_ascen[i]['team1_key']] += 1
#         narrow_losses[matchups_data_by_point_diff_ascen[i]['team2_key']] += 1
#         narrow_matchups[matchups_data_by_point_diff_ascen[i]['team1_key']] += 1
#         narrow_matchups[matchups_data_by_point_diff_ascen[i]['team2_key']] += 1
#     print('')

# # This can be done better
# # It currently just takes who won the most, who lost the most, and who appeared the most in the top 10 (and doesn't deal with ties of these, just shows 1 of them)
# # But this can be turned into who has the best record in matchups decided by less then x (5-10 probably) points
# # Can also see who has the most matchups decided by less then x points
# # i.e. a different award (Best/Worst Close Matchup Record)
# # Award for loser of most blowouts too (50+ pts)? - Might not be that interesting will likely just be people near the bottom of the table
# narrow_wins = sorted(narrow_wins.items(), key=lambda item: item[1], reverse=True)
# narrow_losses = sorted(narrow_losses.items(), key=lambda item: item[1], reverse=True)
# narrow_matchups = sorted(narrow_matchups.items(), key=lambda item: item[1], reverse=True)
# print(f"Wow! {query.get_team_name_from_key(narrow_wins[0][0])} you love winning by the skin of your teeth, you won {narrow_wins[0][1]} close matchups!")
# print(f"Unlucky {query.get_team_name_from_key(narrow_losses[0][0])}, you lost {narrow_losses[0][1]} close matchups :(")
# print(f"{query.get_team_name_from_key(narrow_matchups[0][0])}, you have nerves of steel. You were in {narrow_matchups[0][1]} close matchups!")

In [3]:
metrics = Metrics(query)
await metrics.get_closest_matchups()

{'id': 'closest_matchups',
 'data': [{'rank': 1,
   'image_url': 'https://s.yimg.com/cv/apiv2/default/nhl/nhl_5_t.png',
   'main_text': "Theodore's Poo Poo Hospital (177.1) def. Miami Steamrollers (176.7)",
   'sub_text': '',
   'stat': 0.4},
  {'rank': 2,
   'image_url': 'https://s.yimg.com/cv/apiv2/default/nhl/nhl_1_t.png',
   'main_text': "Tim's Terrific Team (213.2) def. Cyrus's Cursed Team (212.7)",
   'sub_text': '',
   'stat': 0.5},
  {'rank': 3,
   'image_url': 'https://s.yimg.com/cv/apiv2/default/nhl/nhl_6_k.png',
   'main_text': "Kevin's Incredible Team (191.5) def. Miami Steamrollers (191.0)",
   'sub_text': '',
   'stat': 0.5},
  {'rank': 4,
   'image_url': 'https://s.yimg.com/cv/apiv2/default/nhl/nhl_1_t.png',
   'main_text': "Tim's Terrific Team (185.6) def. Eric’s a Bitch Team (184.6)",
   'sub_text': '',
   'stat': 1.0},
  {'rank': 5,
   'image_url': 'https://s.yimg.com/cv/apiv2/default/nhl/nhl_6_k.png',
   'main_text': "Kevin's Incredible Team (167.9) def. Tim's Terrif

In [10]:
'''
Biggest Blowouts (Boat Raced Award)
'''
async def get_biggest_blowout_matchups():
    completed_matchups_data = await query.get_matchup_data()
    top_x = 10
    matchups_data_by_point_diff_descen = sorted(completed_matchups_data, key=lambda x: abs(x['point_diff']), reverse=True)
    biggest_blowouts_list = [
            {
            "rank": i+1,
            "image_url": matchups_data_by_point_diff_descen[i]['team1_url'],
            "main_text": f"{query.get_team_name_from_key(matchups_data_by_point_diff_descen[i]['team1_key'])} ({matchups_data_by_point_diff_descen[i]['team1_points']}) def. {query.get_team_name_from_key(matchups_data_by_point_diff_descen[i]['team2_key'])} ({matchups_data_by_point_diff_descen[i]['team2_points']})",
            "sub_text": "playoffs" if matchups_data_by_point_diff_descen[i]['is_playoffs'] else "",
            "stat": matchups_data_by_point_diff_descen[i]['point_diff']
            } for i in range(top_x)
        ]
    return {"id": "biggest_blowouts", "data": biggest_blowouts_list}

In [4]:
metrics = Metrics(query)
await metrics.get_biggest_blowout_matchups()

{'id': 'biggest_blowouts',
 'data': [{'rank': 1,
   'image_url': 'https://s.yimg.com/cv/apiv2/default/nhl/nhl_5_t.png',
   'main_text': "Theodore's Poo Poo Hospital (262.8) def. Kevin's Incredible Team (124.3)",
   'sub_text': '',
   'stat': 138.5},
  {'rank': 2,
   'image_url': 'https://s.yimg.com/cv/apiv2/default/nhl/nhl_12_k.png',
   'main_text': 'Eric’s a Bitch Team (243.5) def. Miami Steamrollers (123.4)',
   'sub_text': '',
   'stat': 120.1},
  {'rank': 3,
   'image_url': 'https://s.yimg.com/cv/apiv2/default/nhl/nhl_6_e.png',
   'main_text': "eric's Awe-Inspiring Team (209.6) def. adam2 (118.0)",
   'sub_text': '',
   'stat': 91.6},
  {'rank': 4,
   'image_url': 'https://s.yimg.com/cv/apiv2/default/nhl/nhl_9_c.png',
   'main_text': "Cyrus's Cursed Team (200.0) def. Miami Steamrollers (108.4)",
   'sub_text': '',
   'stat': 91.6},
  {'rank': 5,
   'image_url': 'https://s.yimg.com/cv/apiv2/default/nhl/nhl_1_t.png',
   'main_text': "Tim's Terrific Team (198.0) def. Miami Steamroller

In [12]:
'''
Team Dominance Over Another Team (Lopsided Rivalry, Pure Dominance)

- Seems as if the team keys are always ordered the same with the smallest team ID first
  But if this assumption is ever not correct it will completely mess up the stat
  Could add error handling for this scenario but may not be necessary
  NOTE: error handling added because now the teams are reordered to have winning team first (not always the smallest team ID first)

- incl_playoffs and incl_consolation tags could make for some fun statements on the frontend 
  but this does not handle if a team played in multiple playoff games with some being non-consolation and others being consolation
  (but I guess you would only play a team once in the playoffs so if they played playoffs and non-conslation it guarantees there was only 1 matchup and it was real playoffs) - I think this statement is TRUE
  (not sure if teams can play multiple times in consolation, sometimes consolation could be a ladder format where you can go up and down ex. ESPN fantasy)
'''
async def get_rivalry_dominance():
    completed_matchups_data = await query.get_matchup_data()
    matchup_dict = defaultdict(list)
    for matchup in completed_matchups_data:
        key = frozenset([matchup['team1_key'], matchup['team2_key']])
        matchup_dict[key].append(matchup)
    matchup_dict = dict(matchup_dict)
    season_matchup_stats = {}
    for matchup_combination in matchup_dict.keys():
        team1_total_points = 0
        team2_total_points = 0
        for matchup in matchup_dict[matchup_combination]:
            if matchup_combination in season_matchup_stats.keys() and matchup['team1_key'] == season_matchup_stats[matchup_combination]['team2_key']:
                team1_total_points += matchup['team2_points']
                team2_total_points += matchup['team1_points']
                season_matchup_stats[matchup_combination]['team1_total_points'] = round(team1_total_points, 2)
                season_matchup_stats[matchup_combination]['team2_total_points'] = round(team2_total_points, 2)
                season_matchup_stats[matchup_combination] = {
                    'team1_key' : matchup['team2_key'],
                    'team1_name' : matchup['team2_name'],
                    'team1_total_points' : round(team1_total_points, 2),
                    'team1_url' : matchup['team2_url'],
                    'team2_key' : matchup['team1_key'],
                    'team2_name' : matchup['team1_name'],
                    'team2_total_points' : round(team2_total_points, 2),
                    'team2_url' : matchup['team1_url'],
                    'incl_playoffs' : matchup['is_playoffs'],
                    'incl_consolation' :matchup['is_consolation']
                }
            else:
                team1_total_points += matchup['team1_points']
                team2_total_points += matchup['team2_points']
                season_matchup_stats[matchup_combination] = {
                    'team1_key' : matchup['team1_key'],
                    'team1_name' : matchup['team1_name'],
                    'team1_total_points' : round(team1_total_points, 2),
                    'team1_url' : matchup['team1_url'],
                    'team2_key' : matchup['team2_key'],
                    'team2_name' : matchup['team2_name'],
                    'team2_total_points' : round(team2_total_points, 2),
                    'team2_url' : matchup['team2_url'],
                    'incl_playoffs' : matchup['is_playoffs'],
                    'incl_consolation' :matchup['is_consolation']
                }
            season_matchup_stats[matchup_combination]['total_point_diff'] = round(season_matchup_stats[matchup_combination]['team1_total_points'] - season_matchup_stats[matchup_combination]['team2_total_points'], 2)
        if season_matchup_stats[matchup_combination]['total_point_diff'] < 0:
            team1_key_temp = season_matchup_stats[matchup_combination]['team1_key']
            team1_name_temp = season_matchup_stats[matchup_combination]['team1_name']
            team1_total_points_temp = season_matchup_stats[matchup_combination]['team1_total_points']
            team1_url_temp = season_matchup_stats[matchup_combination]['team1_url']
            season_matchup_stats[matchup_combination]['team1_key'] = season_matchup_stats[matchup_combination]['team2_key']
            season_matchup_stats[matchup_combination]['team1_name'] = season_matchup_stats[matchup_combination]['team2_name']
            season_matchup_stats[matchup_combination]['team1_total_points'] = season_matchup_stats[matchup_combination]['team2_total_points']
            season_matchup_stats[matchup_combination]['team1_url'] = season_matchup_stats[matchup_combination]['team2_url']
            season_matchup_stats[matchup_combination]['team2_key'] = team1_key_temp
            season_matchup_stats[matchup_combination]['team2_name'] = team1_name_temp
            season_matchup_stats[matchup_combination]['team2_total_points'] = team1_total_points_temp
            season_matchup_stats[matchup_combination]['team2_url'] = team1_url_temp
            season_matchup_stats[matchup_combination]['total_point_diff'] = abs(season_matchup_stats[matchup_combination]['total_point_diff'])
    season_matchup_stats_descen = sorted(season_matchup_stats.items(), key=lambda matchup: abs(matchup[1]['total_point_diff']), reverse=True)
    top_x = 10
    season_matchup_list = [
            {
            "rank": i+1,
            "image_url": season_matchup_stats_descen[i][1]['team1_url'],
            "main_text": f"{season_matchup_stats_descen[i][1]['team1_name']} ({season_matchup_stats_descen[i][1]['team1_total_points']}) def. {season_matchup_stats_descen[i][1]['team2_name']} ({season_matchup_stats_descen[i][1]['team2_total_points']})",
            "sub_text": "playoffs" if season_matchup_stats_descen[i][1]['incl_playoffs'] else "",
            "stat": season_matchup_stats_descen[i][1]['total_point_diff']
            } for i in range(top_x)
        ]
    return {"id": "rivalry_dominance", "data": season_matchup_list}

# print('Some matchups were never fair from the start')
# print('The most lopsided season matchups were...')
# for i in range(top_x):
#     print(f"{i+1}. {season_matchup_stats_descen[i][1]['team1_name']} def. {season_matchup_stats_descen[i][1]['team2_name']}")
#     print(f"Total Score: {season_matchup_stats_descen[i][1]['team1_total_points']}-{season_matchup_stats_descen[i][1]['team2_total_points']}") 
#     print(f"Total Point Differential: {abs(season_matchup_stats_descen[i][1]['total_point_diff'])}")
#     print(f"Includes Playoffs: {'Yes' if season_matchup_stats_descen[i][1]['incl_playoffs'] else 'No'}")
#     print(f"Includes Consolation: {'Yes' if season_matchup_stats_descen[i][1]['incl_consolation'] else 'No'}")
#     print('')

In [6]:
metrics = Metrics(query)
await metrics.get_rivalry_dominance()

{'id': 'rivalry_dominance',
 'data': [{'rank': 1,
   'image_url': 'https://s.yimg.com/cv/apiv2/default/nhl/nhl_12_k.png',
   'main_text': 'Eric’s a Bitch Team (592.3) def. Miami Steamrollers (381.6)',
   'sub_text': '',
   'stat': 210.7},
  {'rank': 2,
   'image_url': 'https://s.yimg.com/cv/apiv2/default/nhl/nhl_5_t.png',
   'main_text': "Theodore's Poo Poo Hospital (623.7) def. Kevin's Incredible Team (414.7)",
   'sub_text': '',
   'stat': 209.0},
  {'rank': 3,
   'image_url': 'https://s.yimg.com/cv/apiv2/default/nhl/nhl_1_t.png',
   'main_text': "Tim's Terrific Team (587.5) def. Miami Steamrollers (406.7)",
   'sub_text': '',
   'stat': 180.8},
  {'rank': 4,
   'image_url': 'https://s.yimg.com/cv/apiv2/default/nhl/nhl_9_c.png',
   'main_text': "Cyrus's Cursed Team (660.3) def. Kevin's Incredible Team (480.0)",
   'sub_text': '',
   'stat': 180.3},
  {'rank': 5,
   'image_url': 'https://s.yimg.com/cv/apiv2/default/nhl/nhl_6_e.png',
   'main_text': "eric's Awe-Inspiring Team (826.3) d

In [172]:
sorted_season_matchup_stats

[(frozenset({'427.l.97108.t.2', '427.l.97108.t.4'}),
  {'team1_key': '427.l.97108.t.2',
   'team1_name': 'Eric’s a Bitch Team',
   'team1_total_points': 592.3,
   'team2_key': '427.l.97108.t.4',
   'team2_name': 'Miami Steamrollers',
   'team2_total_points': 381.6,
   'total_point_diff': 210.7}),
 (frozenset({'427.l.97108.t.3', '427.l.97108.t.8'}),
  {'team1_key': '427.l.97108.t.3',
   'team1_name': "Theodore's Poo Poo Hospital",
   'team1_total_points': 623.7,
   'team2_key': '427.l.97108.t.8',
   'team2_name': "Kevin's Incredible Team",
   'team2_total_points': 414.7,
   'total_point_diff': 209.0}),
 (frozenset({'427.l.97108.t.4', '427.l.97108.t.6'}),
  {'team1_key': '427.l.97108.t.4',
   'team1_name': 'Miami Steamrollers',
   'team1_total_points': 406.7,
   'team2_key': '427.l.97108.t.6',
   'team2_name': "Tim's Terrific Team",
   'team2_total_points': 587.5,
   'total_point_diff': -180.8}),
 (frozenset({'427.l.97108.t.7', '427.l.97108.t.8'}),
  {'team1_key': '427.l.97108.t.7',
   '

Playground

In [29]:
drops.values()

dict_values([['Matt Boldy', 7], ['Ukko-Pekka Luukkonen', 6], ['Darnell Nurse', 5], ['Jakob Chychrun', 5], ['Jacob Markstrom', 5], ['Brent Burns', 5], ['Claude Giroux', 5], ['Tyler Toffoli', 4], ['Ilya Samsonov', 4], ['Drake Batherson', 4], ['Josh Manson', 4], ['Connor Bedard', 4], ['Logan Thompson', 4], ['Jeff Skinner', 4], ['Tom Wilson', 3], ['Filip Gustavsson', 3], ['Joey Daccord', 3], ['Boone Jenner', 3], ['Alex Lyon', 3], ['Joseph Woll', 3], ['Antti Raanta', 3], ['Jared McCann', 3], ['Elias Lindholm', 3], ['Laurent Brossoit', 3], ['Adam Larsson', 3], ['Rickard Rakell', 3], ['Brayden McNabb', 3], ['Mattias Ekholm', 3], ['William Karlsson', 3], ['Jonathan Quick', 3], ['Carter Hart', 3], ['Owen Tippett', 3], ['Bowen Byram', 2], ['Jake McCabe', 2], ['Thatcher Demko', 2], ['Scott Wedgewood', 2], ['Mark Scheifele', 2], ['Jonas Johansson', 2], ['Jeremy Swayman', 2], ['Gustav Forsling', 2], ['Karel Vejmelka', 2], ['David Rittich', 2], ['Neal Pionk', 2], ['Seth Jarvis', 2], ['Frederik Ander

In [30]:
transaction_type = []
for transaction in transactions:
    transaction_type.append(transaction.type)

set(transaction_type)

{'add', 'add/drop', 'commish', 'drop', 'trade'}

In [31]:
weeks = [1]
','.join(str(week) for week in weeks)

'1'

In [32]:
weeks = [1, 2, 3, 4, 5, 6]
weeks = ','.join(str(week) for week in weeks)
url = f"https://fantasysports.yahooapis.com/fantasy/v2/league/{LEAGUE_KEY}/scoreboard;week={weeks}"
week1 = api_request(url=url)
week1

{'league': {'league_key': '427.l.97108',
  'league_id': '97108',
  'name': 'Men2',
  'url': 'https://hockey.fantasysports.yahoo.com/2023/hockey/97108',
  'logo_url': '',
  'password': '',
  'draft_status': 'postdraft',
  'num_teams': '8',
  'edit_key': '2024-03-31',
  'weekly_deadline': 'intraday',
  'league_update_timestamp': '1711958417',
  'scoring_type': 'headpoint',
  'league_type': 'private',
  'renew': '',
  'renewed': '453_67269',
  'felo_tier': 'bronze',
  'iris_group_chat_id': '',
  'short_invitation_url': 'https://hockey.fantasysports.yahoo.com/2023/hockey/97108/invitation?key=3ce97a4f81b53dce&ikey=3860fd06c80066cd',
  'allow_add_to_dl_extra_pos': '1',
  'is_pro_league': '0',
  'is_cash_league': '0',
  'current_week': '24',
  'start_week': '1',
  'start_date': '2023-10-10',
  'end_week': '24',
  'end_date': '2024-03-31',
  'is_finished': '1',
  'is_plus_league': '0',
  'game_code': 'nhl',
  'season': '2023',
  'scoreboard': {'week': '1,2,3,4,5,6',
   'matchups': [{'week': '1

In [33]:
week1['league']['scoreboard']['matchups'][3]['teams'][0]['team_points']['total']

'182.60'

In [34]:
week1['league']['scoreboard']['matchups'][3]

{'week': '1',
 'week_start': '2023-10-10',
 'week_end': '2023-10-15',
 'status': 'postevent',
 'is_playoffs': '0',
 'is_consolation': '0',
 'is_tied': '0',
 'winner_team_key': '427.l.97108.t.6',
 'stat_winners': [{'stat_id': '1', 'winner_team_key': '427.l.97108.t.6'},
  {'stat_id': '2', 'winner_team_key': '427.l.97108.t.7'},
  {'stat_id': '4', 'winner_team_key': '427.l.97108.t.6'},
  {'stat_id': '8', 'winner_team_key': '427.l.97108.t.7'},
  {'stat_id': '11', 'winner_team_key': '427.l.97108.t.7'},
  {'stat_id': '14', 'winner_team_key': '427.l.97108.t.7'},
  {'stat_id': '31', 'winner_team_key': '427.l.97108.t.6'},
  {'stat_id': '32', 'winner_team_key': '427.l.97108.t.6'},
  {'stat_id': '19', 'is_tied': '1'},
  {'stat_id': '22', 'is_tied': '1'},
  {'stat_id': '25', 'winner_team_key': '427.l.97108.t.6'},
  {'stat_id': '27', 'is_tied': '1'}],
 'teams': [{'team_key': '427.l.97108.t.6',
   'team_id': '6',
   'name': "Tim's Terrific Team",
   'url': 'https://hockey.fantasysports.yahoo.com/2023

In [35]:
week1

{'league': {'league_key': '427.l.97108',
  'league_id': '97108',
  'name': 'Men2',
  'url': 'https://hockey.fantasysports.yahoo.com/2023/hockey/97108',
  'logo_url': '',
  'password': '',
  'draft_status': 'postdraft',
  'num_teams': '8',
  'edit_key': '2024-03-31',
  'weekly_deadline': 'intraday',
  'league_update_timestamp': '1711958417',
  'scoring_type': 'headpoint',
  'league_type': 'private',
  'renew': '',
  'renewed': '453_67269',
  'felo_tier': 'bronze',
  'iris_group_chat_id': '',
  'short_invitation_url': 'https://hockey.fantasysports.yahoo.com/2023/hockey/97108/invitation?key=3ce97a4f81b53dce&ikey=3860fd06c80066cd',
  'allow_add_to_dl_extra_pos': '1',
  'is_pro_league': '0',
  'is_cash_league': '0',
  'current_week': '24',
  'start_week': '1',
  'start_date': '2023-10-10',
  'end_week': '24',
  'end_date': '2024-03-31',
  'is_finished': '1',
  'is_plus_league': '0',
  'game_code': 'nhl',
  'season': '2023',
  'scoreboard': {'week': '1,2,3,4,5,6',
   'matchups': [{'week': '1

In [36]:
scoreboard = Scoreboard(week1['league']['scoreboard'])

In [37]:
scoreboard.matchups()

KeyError: 'matchup'

In [ ]:
query_week1 = query.get_league_scoreboard_by_week(chosen_week=1)
query_week1

Scoreboard({
  "matchups": [
    {
      "matchup": {
        "is_consolation": 0,
        "is_playoffs": 0,
        "is_tied": 0,
        "status": "postevent",
        "teams": [
          {
            "team": {
              "clinched_playoffs": 1,
              "draft_position": 7,
              "has_draft_grade": 0,
              "is_owned_by_current_login": 1,
              "league_scoring_type": "headpoint",
              "managers": {
                "manager": {
                  "email": "amtadam14@hotmail.ca",
                  "felo_score": 577,
                  "felo_tier": "bronze",
                  "guid": "NGSXRXJK2U3UJVTCC2PO3L2A6M",
                  "image_url": "https://s.yimg.com/ag/images/default_user_profile_pic_64sq.jpg",
                  "is_commissioner": 0,
                  "is_current_login": 1,
                  "manager_id": 1,
                  "nickname": "adam"
                }
              },
              "name": "adam2",
              "number_

In [ ]:
query_week1.clean_data_dict()['matchups'][0]

{'matchup': Matchup({
   "is_consolation": 0,
   "is_playoffs": 0,
   "is_tied": 0,
   "status": "postevent",
   "teams": [
     {
       "team": {
         "clinched_playoffs": 1,
         "draft_position": 7,
         "has_draft_grade": 0,
         "is_owned_by_current_login": 1,
         "league_scoring_type": "headpoint",
         "managers": {
           "manager": {
             "email": "amtadam14@hotmail.ca",
             "felo_score": 577,
             "felo_tier": "bronze",
             "guid": "NGSXRXJK2U3UJVTCC2PO3L2A6M",
             "image_url": "https://s.yimg.com/ag/images/default_user_profile_pic_64sq.jpg",
             "is_commissioner": 0,
             "is_current_login": 1,
             "manager_id": 1,
             "nickname": "adam"
           }
         },
         "name": "adam2",
         "number_of_moves": 23,
         "number_of_trades": 0,
         "roster_adds": {
           "coverage_type": "week",
           "coverage_value": 26,
           "value": 0
   

In [ ]:
query_week1.matchups[0].teams[0].points

140.9

In [ ]:
['teamA', 'teamB'] == ['teamB', 'teamA']

False

In [ ]:
frozenset(['teamA', 'teamB']) == frozenset(['teamB', 'teamA'])

True

In [107]:
frozenset([('teamA', 'teamB'), ('teamB', 'teamA'), ('teamA', 'teamB')])

frozenset({('teamA', 'teamB'), ('teamB', 'teamA')})

In [5]:
url = f"/league/{query.league_key};out=standings,settings"
response = await query.get_response(url)

In [8]:
weeks = ','.join(str(week) for week in range(query.league_start_week, query.league_end_week+1))
url = f"/league/{query.league_key}/scoreboard;week={weeks}"
response = await query.get_response(url)
scoreboard = response['league']['scoreboard']['matchups']

In [9]:
scoreboard

[{'week': '1',
  'week_start': '2023-10-10',
  'week_end': '2023-10-15',
  'status': 'postevent',
  'is_playoffs': '0',
  'is_consolation': '0',
  'is_tied': '0',
  'winner_team_key': '427.l.97108.t.1',
  'stat_winners': [{'stat_id': '1', 'winner_team_key': '427.l.97108.t.1'},
   {'stat_id': '2', 'winner_team_key': '427.l.97108.t.1'},
   {'stat_id': '4', 'winner_team_key': '427.l.97108.t.8'},
   {'stat_id': '8', 'winner_team_key': '427.l.97108.t.1'},
   {'stat_id': '11', 'winner_team_key': '427.l.97108.t.1'},
   {'stat_id': '14', 'winner_team_key': '427.l.97108.t.8'},
   {'stat_id': '31', 'winner_team_key': '427.l.97108.t.1'},
   {'stat_id': '32', 'winner_team_key': '427.l.97108.t.1'},
   {'stat_id': '19', 'is_tied': '1'},
   {'stat_id': '22', 'winner_team_key': '427.l.97108.t.1'},
   {'stat_id': '25', 'winner_team_key': '427.l.97108.t.8'},
   {'stat_id': '27', 'winner_team_key': '427.l.97108.t.8'}],
  'teams': [{'team_key': '427.l.97108.t.1',
    'team_id': '1',
    'name': 'adam2',
 